# Accelerate BERT & RoBERTa inference with DeepSpeed-Inference

In this session, you will learn how to optimize Hugging Face Transformers models for GPU inference using [DeepSpeed-Inference](https://www.deepspeed.ai/tutorials/inference-tutorial/). The session will show you how to apply state-of-the-art optimization techniques using [DeepSpeed-Inference](https://www.deepspeed.ai/tutorials/inference-tutorial/). 
This session will focus on single GPU inference on BERT and RoBERTa models. 
By the end of this session, you will know optimize your Hugging Face Transformers models (BERT, RoBERTa) using DeepSpeed-Inference. We are going to optimize a DistilBERT model for Question Answering, which was fine-tuned on the SQuAD dataset to decrease the latency from 7ms to 3ms for a sequence lenght of 128.

You will learn how to:
1. Setup Development Environment
2. Load vanilla BERT model and set baseline
3. Optimize BERT for GPU using DeepSpeeds `InferenceEngine`
4. Evaluate the performance and speed

Let's get started! 🚀

_This tutorial was created and run on an g4dn.xlarge AWS EC2 Instance including a NVIDIA T4._

---

## Quick Intro: What is DeepSpeed-Inference

[DeepSpeed-Inference](https://www.deepspeed.ai/tutorials/inference-tutorial/) is an extension of the [DeepSpeed](https://www.deepspeed.ai/) framework focused on inference workloads.  [DeepSpeed Inference](https://www.deepspeed.ai/#deepspeed-inference) combines model parallelism technology such as tensor, pipeline-parallelism, with custom optimzed cuda kernels.
DeepSpeed provides a seamless inference mode for compatible transformer based models trained using DeepSpeed, Megatron, and HuggingFace. For list of compatible models please see [here](https://github.com/microsoft/DeepSpeed/blob/master/deepspeed/module_inject/replace_policy.py).
As mentioned DeepSpeed-Inference integrates model-parallelism techniques allowing you so run multi-GPU inference for LLM, like [BLOOM](https://huggingface.co/bigscience/bloom) with 176 billion parameters.
If you want to learn more about DeepSpeed inference: 
* [Paper: DeepSpeed Inference: Enabling Efficient Inference of Transformer Models at Unprecedented Scale](https://arxiv.org/pdf/2207.00032.pdf)
* [Blog: Accelerating large-scale model inference and training via system optimizations and compression](https://www.microsoft.com/en-us/research/blog/deepspeed-accelerating-large-scale-model-inference-and-training-via-system-optimizations-and-compression/)


## 1. Setup Development Environment

Our first step is to install Deepspeed, along with PyTorch, Transfromers and some other libraries. Running the following cell will install all the required packages.

_Note: You need a machine with a GPU and a compatible CUDA installed. You can check this by running `nvidia-smi` in your terminal. If you have a correct environment you should statistics abour your GPU._

In [28]:
!pip install torch==1.12.1 torchvision --extra-index-url https://download.pytorch.org/whl/cu113
!DS_BUILD_TRANSFORMER_INFERENCE=1  pip install deepspeed
!pip install transformers[sentencepiece]==4.21.1
!pip install datasets evaluate[evaluator]==0.2.2 seqeval

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB ? eta 0:00:00 0:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.4/23.4 MB 38.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.12.0+cu113
    Uninstalling torchvision-0.12.0+cu113:
      Successfully uninstalled torchvision-0.12.0+cu113
You should consider upgrading via the '/usr/bin/python3.8 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should

Before we start. Lets make sure we all packages are installed correctly

In [30]:
import re
import torch 

# check deepspeed installation
report = !python3 -m deepspeed.env_report
r = re.compile('.*transformer_inference.*YES.*')
assert any(r.match(line) for line in report) == True, "DeepSpeed Inference not correct installed"

# check cuda and torch version
torch_version, cuda_version = torch.__version__.split("+")
torch_version = ".".join(torch_version.split(".")[:2])
cuda_version = f"{cuda_version[2:4]}.{cuda_version[4:]}"
r = re.compile(f'.*torch.*{torch_version}.*')
assert any(r.match(line) for line in report) == True, "Wrong Torch version"
r = re.compile(f'.*cuda.*{cuda_version}.*')
assert any(r.match(line) for line in report) == True, "Wrong Cuda version"

## 2. Load vanilla BERT model and set baseline

After we setup our environment we create a baseline for our model. We use the [dslim/bert-large-NER](https://huggingface.co/dslim/bert-large-NER) a fine-tuned BERT-large model on on the English version of the standard [CoNLL-2003 Named Entity Recognition](https://www.aclweb.org/anthology/W03-0419.pdf) dataset achieving an f1 score `95.7%`.

To create our baseline we load the model with `transformers` and create a `token-classification` pipeline.

In [31]:
from transformers import AutoTokenizer, AutoModelForTokenClassification,pipeline
from transformers import pipeline

# Model Repository on huggingface.co
model_id="dslim/bert-large-NER"

# Load Model and Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForTokenClassification.from_pretrained(model_id)

# Create a pipeline for token classification
token_clf = pipeline("token-classification", model=model, tokenizer=tokenizer,device=0)

# Text pipeline
example = "My name is Wolfgang and I live in Berlin"
ner_results = token_clf(example)
print(ner_results)


[{'entity': 'B-PER', 'score': 0.9971501, 'index': 4, 'word': 'Wolfgang', 'start': 11, 'end': 19}, {'entity': 'B-LOC', 'score': 0.9986046, 'index': 9, 'word': 'Berlin', 'start': 34, 'end': 40}]


Create Baseline with `evaluate` using the `evaluator` and the `conll2003` dataset. The Evaluator class allows us to evaluate a model/pipeline on a dataset using a defined metric. 

In [12]:
from evaluate import evaluator
from datasets import load_dataset

# load eval dataset
eval_dataset = load_dataset("conll2003", split="validation")

# define evaluator
task_evaluator = evaluator("token-classification")

# run baseline
results = task_evaluator.compute(
    model_or_pipeline=token_clf,
    data=eval_dataset,
    metric="seqeval",
)

print(f"Overall f1 score for our model is {results['overall_f1']*100:.2f}")
print(f"The avg. Latency of the model is {results['latency_in_seconds']*1000:.2f}ms")

Reusing dataset conll2003 (/home/ubuntu/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98)
Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98/cache-b6771e194accd0b2.arrow


Overall f1 score for our model is 95.7593027153872.2f
Overall f1 score for our model is 95.7593027153872.2f


Our model achieves an f1 score of `95.8%` on the CoNLL-2003 dataset with a average latency across the dataset of `18.9ms`. 

## 3. Optimize BERT for GPU using DeepSpeeds `InferenceEngine`

init: https://deepspeed.readthedocs.io/en/latest/inference-init.html

In [32]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification,pipeline
from transformers import pipeline
from deepspeed.module_inject import HFBertLayerPolicy
import deepspeed

# load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForTokenClassification.from_pretrained(model_id)

# init deepspeed inference engine
ds_model = deepspeed.init_inference(
    model=model,      # Transformers models
    mp_size=1,        # Number of GPU
    dtype=torch.half, # dtype of the weights (fp16)
    # injection_policy={"BertLayer" : HFBertLayerPolicy}, # replace BertLayer with DS HFBertLayerPolicy
    replace_method="auto",
    replace_with_kernel_inject=True, # replace the model with the kernel injector
)



ds_clf = pipeline("token-classification", model=ds_model, tokenizer=tokenizer,device=0)
example = "My name is Wolfgang and I live in Berlin"

ner_results = ds_clf(example)
print(ner_results)



[2022-08-15 15:15:52,979] [INFO] [logging.py:68:log_dist] [Rank -1] DeepSpeed info: version=0.7.0, git-hash=unknown, git-branch=unknown
[2022-08-15 15:15:52,980] [INFO] [logging.py:68:log_dist] [Rank -1] quantize_bits = 8 mlp_extra_grouping = False, quantize_groups = 1


ImportError: /home/ubuntu/.local/lib/python3.8/site-packages/deepspeed/ops/transformer/inference/transformer_inference_op.cpython-38-x86_64-linux-gnu.so: undefined symbol: curandCreateGenerator

In [ ]:
ds_model

InferenceEngine(
  (module): BertForTokenClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(28996, 1024, padding_idx=0)
        (position_embeddings): Embedding(512, 1024)
        (token_type_embeddings): Embedding(2, 1024)
        (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=1024, out_features=1024, bias=True)
                (key): Linear(in_features=1024, out_features=1024, bias=True)
                (value): Linear(in_features=1024, out_features=1024, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=1024, out_features=10

In [19]:
# run baseline
results = task_evaluator.compute(
    model_or_pipeline=ds_clf,
    data=eval_dataset,
    metric="seqeval",
)

print(f"Overall f1 score for our model is {results['overall_f1']*100:.2f}")
print(f"The avg. Latency of the model is {results['latency_in_seconds']*1000:.2f}ms")

Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98/cache-b6771e194accd0b2.arrow


Overall f1 score for our model is 95.76
The avg. Latency of the model is 18.90ms
